In [1]:
import os 
import json 
import pdb 
from glob import glob 

import numpy as np 
import pandas as pd

from doctr.models import ocr_predictor

from doctr.io import DocumentFile
import matplotlib.pyplot as plt

import cv2

ModuleNotFoundError: No module named 'doctr.models'; 'doctr' is not a package

In [ ]:
searchstring = os.path.join(os.getcwd(), 'data/processed/*.jpg' )
paths = glob(searchstring)


path = paths[2]
# path = './data/processed/JK17-04_0.jpg'
filename = path.split("/")[-1].split('.jpg')[0]
folderpath = os.path.dirname(path)
# detection_image_path = os.path.join('./data/ocr/detection', filename + '.jpg')
extracted_info_path = os.path.join('./data/ocr/extraction', filename + '.json')
synthesised_image_path = os.path.join('./data/ocr/synthesis', filename + '.jpg')


: 

In [ ]:
image = cv2.imread(path)
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

# gaussian smoothing
gray = cv2.GaussianBlur(gray, (3,3), 0)

grad = np.absolute(gray)
(minVal, maxVal) = (np.min(grad), np.max(grad))
grad = (grad - minVal) / (maxVal - minVal)
grad = (grad * 255).astype("uint8")

plt.figure(figsize=(10,8))
plt.imshow(grad)

cwd = os.getcwd()
os.chdir(folderpath)
cv2.imwrite(filename + '.jpg',grad)

os.chdir(cwd)

: 

In [ ]:




doc = DocumentFile.from_images(path)

model = ocr_predictor(det_arch='db_resnet50', reco_arch='crnn_vgg16_bn', pretrained=True, 
                    assume_straight_pages=True, 
                    export_as_straight_boxes=True)

result = model(doc)

result.show(doc)


: 

In [ ]:
# page simulated with extracted text
synthetic_pages = result.synthesize()


synth_arr = synthetic_pages[0].astype(np.uint8)
plt.figure(figsize=(10,6))
plt.imshow(synth_arr , aspect='auto', cmap='gray')
plt.axis('off')
plt.show()

# json export 
json_output = result.export()
with open(extracted_info_path,'w') as f :
    json.dump(json_output,f)
    f.close()

: 

## Extracted Text to DF

In [ ]:
import json 
import os 
import pdb 
import numpy as np 
import pandas as pd 

from sklearn.cluster import AgglomerativeClustering

sample_extraction_path = './data/ocr/extraction/JK17-15_47.json'

In [ ]:
extracted_info_file = open(sample_extraction_path,'r')
extracted_info = json.load(extracted_info_file)

In [ ]:
df = pd.read_json(sample_extraction_path)

In [ ]:
df.head

<bound method NDFrame.head of                                                pages
0  {'page_idx': 0, 'dimensions': [2519, 1824], 'o...>

In [ ]:
blocks = extracted_info['pages'][0]['blocks']
len(blocks)

11

In [ ]:
len(blocks)

11

In [ ]:
# blocks df 
# lines df 
# words df 
block_df_columns = ['xmin','ymin','xmax','ymax', 'block_num', 'nlines']
lines_df_columns =  ['xmin','ymin','xmax','ymax', 'block_num', 'line_num', 'nwords']
words_df_columns =  ['xmin','ymin','xmax','ymax', 'block_num','line_num', 'word_num','value','confidence']

blocks_df = pd.DataFrame(columns = block_df_columns)
lines_df = pd.DataFrame(columns = lines_df_columns)
words_df = pd.DataFrame(columns =words_df_columns)

for  i, block  in enumerate(blocks) : 
    # update blocks df 

    data = [
         block['geometry'][0][0],
         block['geometry'][0][1],
         block['geometry'][1][0],
         block['geometry'][1][1],
        i,
        len(block['lines'])
    ]

    data_dict = {k : [v] for (k,v) in zip(block_df_columns, data)}
    block_df_ = pd.DataFrame.from_dict(data_dict)
    
    blocks_df = pd.concat([blocks_df, block_df_], ignore_index=True)
    
    # update lines df 
    print(len(block['lines']))
    for j, line in enumerate(block['lines']) : 
        lines_data = [
             line['geometry'][0][0],
             line['geometry'][0][1],
             line['geometry'][1][0],
             line['geometry'][1][1],
            i, j,
            len(line['words'])
        ] 
        lines_data_dict = {k:[v] for (k,v) in zip(lines_df_columns, lines_data)}
        lines_df_ = pd.DataFrame.from_dict(lines_data_dict)

        lines_df = pd.concat([lines_df, lines_df_],  ignore_index=True)
        
    # update words df 
        for k,word in enumerate(line['words']) : 
            
            words_data = [
             word['geometry'][0][0],
             word['geometry'][0][1],
             word['geometry'][1][0],
             word['geometry'][1][1],
             i, j,k,
            word['value'], word['confidence']
        ]
            words_data_dict= {k:[v] for (k,v) in zip(words_df_columns, words_data)}
            words_df_ = pd.DataFrame.from_dict(words_data_dict)

            words_df = pd.concat([words_df, words_df_], ignore_index=True)  
            
            
            
            
            
    

1
1
1
50
1
1
50
17
1
18
3


In [ ]:
blocks_df

,xmin,ymin,xmax,ymax,block_num,nlines
0,0.393456,0.081055,0.636215,0.097656,0,1
1,0.141256,0.115234,0.210037,0.126953,1,1
2,0.327371,0.120117,0.416383,0.131836,2,1
3,0.133164,0.126953,0.539111,0.789062,3,50
4,0.652399,0.125977,0.768384,0.141602,4,1
5,0.556644,0.138672,0.866836,0.155273,5,1
6,0.412337,0.160156,0.506743,0.829102,6,50
7,0.525625,0.510742,0.718483,0.791992,7,17
8,0.636215,0.514648,0.717135,0.526367,8,1
9,0.754897,0.515625,0.818284,0.798828,9,18


In [ ]:
blocks_df['nlines'].sum()

144

In [ ]:
words_df.sort_values(by=['line_num','word_num'], ascending=[True,True], inplace=True)
words_df = words_df[words_df['block_num'].isin([3,6,7,9,10])]
x = pd.pivot_table(index=['line_num'], columns=['block_num','word_num'], fill_value='',data=words_df, values='value',aggfunc= lambda x : ' '.join(x))
x

block_num   3                                   6       7                     \
word_num     0        1         2          3     0       0        1        2   
line_num                                                                       
0          No.  EASTING  NORTHING  ELEVATION  1269  417661                     
1          943   417048   3737943             1251  417674  3737631            
2          944   417070   3737932             1236  417687  3737616            
3          945   417093   3737921             1225  417709  3737616            
4          946   417115   3737910             1223  417746  3737602            
5          947   417137   3737899             1211  417763  3737592            
6          948   417160   3737887             1195  417790  3737578            
7          949   417182   3737876                o  417813  3737570            
8          950   417205   3737865                o  417835  3737560            
9          951   417227   3737854                o  417852  3737549            
10         952   417249   3737843                o  417867  3737544            
11         953   417272   3737832                o  417948                     
12         954   417294   3737821                o  417955  3737474            
13         955   417317   3737810                o  418012  3737476            
14         956   417339   3737799                o  418033   418054  3737468   
15         957   417361   3737788                o  418090                     
16         958   417384   3737777                o  418112  3737392            
17         959   417406   3737766                o                             
18         960   417429   3737755                o                             
19         961   417451   3737744                o                             
20         962   417473   3737732                o                             
21         963   417496   3737721                o                             
22         964   417518   3737710             1083                             
23         965   417541   3737699             1081                             
24         966   417563   3737688             1075                             
25         967   417585   3737677             1075                             
26         968   417608   3737666             1081                             
27         969   417630   3737655             1086                             
28         970   417653   3737644             1097                             
29         971   417675   3737633             1101                             
30         972   417697   3737622             1103                             
31         973   417720   3737611             1106                             
32         974   417742   3737600             1108                             
33         975   417765   3737589             1109                             
34         976   417787   3737577             1110                             
35         977   417809   3737566             1111                             
36         978   417832   3737555             1112                             
37         979   417854   3737544             1109                             
38         980   417877   3737533             1106                             
39         981   417899   3737522             1108                             
40         982   417921   3737511             1120                             
41         983   417944   3737500             1125                             
42         984   417966   3737489             1131                             
43         985   417989   3737478             1132                             
44         986   418011   3737467             1141                             
45         987   418033   3737456             1153                             
46         988   418056   3737445             1154                             


In [ ]:
words_df['xmin_apprx'] = words_df['xmin'].apply(lambda x : round(x,2))
words_df['ymin_apprx'] = words_df['ymin'].apply(lambda x : round(x,2))

y = pd.pivot_table(index=['ymin_apprx'], columns=['xmin_apprx'], fill_value='',data=words_df, values='value',aggfunc= lambda x : ' '.join(x))
y.head(66)

xmin_apprx,0.13,0.14,0.15,0.16,0.20,0.21,0.22,0.23,0.30,0.31,...,0.46,0.47,0.48,0.53,0.54,0.63,0.64,0.75,0.76,0.77
ymin_apprx,,,,,,,,,,,,,,,,,,,,,
0.13,,,,No.,,,,EASTING,,,...,,,,,,,,,,
0.15,,,,943,,,,,,,...,,,,,,,,,,
0.16,,,,,,,,417048,,,...,1269,,,,,,,,,
0.17,,,,944,,,417070,,,,...,,,,,,,,,,
0.18,,,,945,,,,417093,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.75,989,,,,418078,,,,3737434,,...,,,,,,,,,,
0.76,990,,,,418101,,,,3737422,,...,,,,,,,,,,
0.77,991,,,,418123,,,,3737411,,...,,,,418090,,3737392,,1151,,


In [ ]:
# x.loc[:,x.columns[-1]] = x.loc[:,x.columns[-1]].shift(1)

In [ ]:
# x[1:].to_csv('./data/results/96JK17-15_elevation_survey_2.csv')

In [ ]:
from sklearn.cluster import AgglomerativeClustering
clustering = AgglomerativeClustering(
        n_clusters=None,
        metric="manhattan",
        linkage="complete",
        distance_threshold=0.1)

columns = np.setdiff1d(df.columns,['xmin'], assume_unique=True)
X = words_df[columns]
y = pd.Series(words_df['xmin'], index=words_df.index)
clustering.fit(X,y)
    # initialize our list of sorted clusters
sortedClusters = []

KeyError: "None of [Index(['pages'], dtype='object')] are in the [columns]"

In [ ]:
from rich import inspect 
inspect(clustering)

╭──────────────── <class 'sklearn.cluster._agglomerative.AgglomerativeClustering'> ─────────────────╮
│ Agglomerative Clustering.                                                                         │
│                                                                                                   │
│ ╭───────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ AgglomerativeClustering(distance_threshold=0.1, linkage='complete',                           │ │
│ │ │   │   │   │   │   │   metric='manhattan', n_clusters=None)                                  │ │
│ ╰───────────────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                                   │
│                affinity = 'deprecated'                                                            │
│               children_ = array([[ 39,  46],                                                      │
│                                  [235, 239],                                                      │
│                                  [ 19,  27],                                                      │
│                                  [ 41,  48],                                                      │
│                                  [ 76,  83],                                                      │
│                                  [211, 215],                                                      │
│                                  [199, 203],                                                      │
│                                  [163, 167],                                                      │
│                                  [155, 159],                                                      │
│                                  [109, 115],                                                      │
│                                  [ 53,  60],                                                      │
│                                  [  4,  12],                                                      │
│                                  [ 17,  25],                                                      │
│                                  [117, 123],                                                      │
│                                  [ 89,  96],                                                      │
│                                  [227, 231],                                                      │
│                                  [219, 223],                                                      │
│                                  [187, 191],                                                      │
│                                  [179, 183],                                                      │
│                                  [171, 175],                                                      │
│                                  [147, 151],                                                      │
│                                  [131, 135],                                                      │
│                                  [122, 127],                                                      │
│                                  [ 67,  74],                                                      │
│                                  [ 94, 101],                                                      │
│                                  [ 62,  69],                                                      │
│                                  [  2,  10],                                                      │
│                                  [  3,  11],                                                      │
│                                  [110, 116],                                                      │
│                                  [103, 111],                                                      │
│                     